In [ ]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
%matplotlib inline

In [ ]:
t_titre = 35
t_label = 25
fig_width = 15

In [ ]:
!ls data

In [ ]:
observer = 'LB'
mode = 'enregistrement'
from aSPEM import Analysis
e = Analysis()#observer=observer, mode=mode)

In [ ]:
fig, axs = e.plot_velocity(trials=[19, 71])

In [ ]:
fig, axs= e.plot_Fit()

In [ ]:
fig, axs = e.plot_Fit(plot='velocity', trials=[19, 71])

In [ ]:
fig, axs, result = e.plot_Fit(plot='velocity', report=True, trials=[19,71,83])

In [ ]:
for x in range(len(result)) :
    print('essai',x ,'\n', result[0])

### Fit

In [ ]:
# pour générer les paramètre fit ainsi que les figures de tout les essais
#e.Fit()